In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import re 
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['test.ft.txt.bz2', 'train.ft.txt.bz2']


In [2]:
import bz2
train_file = bz2.BZ2File('../input/train.ft.txt.bz2')
test_file = bz2.BZ2File('../input/test.ft.txt.bz2')

train_lines = train_file.readlines()
test_lines = test_file.readlines()

In [3]:
print(len(train_lines), len(test_lines))
#2 indicates a positive review and 1 indicates a negative review
print(train_lines[0])
print(train_lines[10])


3600000 400000
b'__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n'
b"__label__1 The Worst!: A complete waste of time. Typographical errors, poor grammar, and a totally pathetic plot add up to absolutely nothing. I'm embarrassed for this author and very disappointed I actually paid for this book.\n"


In [4]:
train_lines = [x.decode('utf-8') for x in train_lines]
test_lines = [x.decode('utf-8') for x in test_lines]

In [5]:
import re
from tqdm import tqdm

def pre_process(lines):
    reviews = []
    labels = []
    for line in tqdm(lines):
#         parse the review and label
        line = line.split('__label__')[1] 
        label = line[0]
        review = line[1:]
        review = review.lower()
        review = review.split()
#         ps = PorterStemmer()
#         review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
        review = ' '.join(review)
#         sentence=temp[1:len(temp)]
#         sentence[len(sentence)-1]=sentence[len(sentence)-1][0:len(sentence[len(sentence)-1])-3]
        
        reviews.append(review)
        labels.append(label)
        
    return reviews, labels
        
reviews_train, labels_train = pre_process(train_lines)
reviews_test, labels_test = pre_process(test_lines)
    





100%|██████████| 400000/400000 [00:03<00:00, 116982.52it/s]


In [6]:
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

In [7]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(reviews_train)

Using TensorFlow backend.


In [8]:
token_train = tokenizer.texts_to_sequences(reviews_train)
token_test = tokenizer.texts_to_sequences(reviews_test)

In [9]:
maxlen = 128
from keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(token_train, maxlen=128, padding='post')
x_test = pad_sequences(token_test, maxlen=128, padding='post')

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
model = Sequential()
model.add(Embedding(1000, 64,  input_length = x_train.shape[1]))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv1D(32, 7, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(Conv1D(2, 1))
model.add(GlobalAveragePooling1D())
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 128, 64)           64000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 128, 64)           256       
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 128, 32)           14368     
_________________________________________________________________
batch_normalization_20 (Batc (None, 128, 32)           128       
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 128, 32)           3104      
_________________________________________________________________
batch_normalization_21 (Batc (None, 128, 32)           128       
__________

In [38]:
print(labels_train.shape)
y_train = []
for l in labels_train:
    if l == '1':
        l = [0,1]
    else:
        l = [1,0]
    y_train.append(l)
y_train = np.array(y_train)
print(y_train)

(3600000,)
[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [1 0]]


In [39]:
print(labels_test.shape)
print(labels_test)
y_test = []
for l in labels_test:
    if l == '1':
        l = [0,1]
    else:
        l = [1,0]
    y_test.append(l)
y_test = np.array(y_test)
print(y_test)

(400000,)
['2' '2' '1' ... '1' '2' '1']
[[1 0]
 [1 0]
 [0 1]
 ...
 [0 1]
 [1 0]
 [0 1]]


In [40]:
size = 200000
model.fit(x_train, y_train, batch_size=2048, epochs=5, validation_split=0.1)

Train on 3240000 samples, validate on 360000 samples
Epoch 1/5
3240000/3240000 [==============================] - 170s 53us/step - loss: 0.2953 - acc: 0.8787 - val_loss: 0.3197 - val_acc: 0.8628
Epoch 2/5
3240000/3240000 [==============================] - 170s 53us/step - loss: 0.2364 - acc: 0.9045 - val_loss: 0.2555 - val_acc: 0.8910
Epoch 3/5
3240000/3240000 [==============================] - 170s 53us/step - loss: 0.2253 - acc: 0.9100 - val_loss: 0.2210 - val_acc: 0.9107
Epoch 4/5
3240000/3240000 [==============================] - 171s 53us/step - loss: 0.2184 - acc: 0.9133 - val_loss: 0.2375 - val_acc: 0.9071
Epoch 5/5
3240000/3240000 [==============================] - 171s 53us/step - loss: 0.2135 - acc: 0.9158 - val_loss: 0.1998 - val_acc: 0.9226


In [ ]:
model.evaluate (x_test, y_test)


   576/400000 [..............................] - ETA: 1:20 